In [9]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

In [13]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [12]:
from dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# アリとハチの画像へのファイルパスのリストを作成する
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Datasetを作成する
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')


# DataLoaderを作成する
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


In [14]:

# 학습된 vgg-16 모델 로드

# VGG-16 모델의 인스턴스 작성
use_pretrained = True  # 학습된 파라미터 사용
net = models.vgg16(pretrained=use_pretrained)

# VGG16의 마지막 출력층의 출력 유닛을 개미와 벌 두 개로 변경
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 훈련 모드로 설정
net.train()

print('네트워크 설정 완료')

네트워크 설정 완료


In [15]:
# 손실함수 정의
criterion = nn.CrossEntropyLoss()

In [16]:
print(net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [19]:
net.named_parameters

<bound method Module.named_parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kerne

In [20]:
# 파인튜닝으로 학습할 파라미터를 params_to_update 변수의 1~3에 저장한다.
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 학습시킬 층의 파라미터명 지정
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# 파라미터를 각 리스트에 저장
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1에 저장：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2저장：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3저장：", name)

    else:
        param.requires_grad = False
        print("경사계산없음, 학습하지 않음：", name)

params_to_update_1에 저장： features.0.weight
params_to_update_1에 저장： features.0.bias
params_to_update_1에 저장： features.2.weight
params_to_update_1에 저장： features.2.bias
params_to_update_1에 저장： features.5.weight
params_to_update_1에 저장： features.5.bias
params_to_update_1에 저장： features.7.weight
params_to_update_1에 저장： features.7.bias
params_to_update_1에 저장： features.10.weight
params_to_update_1에 저장： features.10.bias
params_to_update_1에 저장： features.12.weight
params_to_update_1에 저장： features.12.bias
params_to_update_1에 저장： features.14.weight
params_to_update_1에 저장： features.14.bias
params_to_update_1에 저장： features.17.weight
params_to_update_1에 저장： features.17.bias
params_to_update_1에 저장： features.19.weight
params_to_update_1에 저장： features.19.bias
params_to_update_1에 저장： features.21.weight
params_to_update_1에 저장： features.21.bias
params_to_update_1에 저장： features.24.weight
params_to_update_1에 저장： features.24.bias
params_to_update_1에 저장： features.26.weight
params_to_update_1에 저장： features.26.bias


In [24]:
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

In [25]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    
    # GPU 사용
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치：", device)

    # 모델을 gpu에 탑재
    net.to(device)

    # 네트워크가 고정되면 고속화 시킴
    torch.backends.cudnn.benchmark = True

    # epoch 루프
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch별 훈련 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 훈련 모드로
            else:
                net.eval()   # 검증 모드로

            epoch_loss = 0.0  # epoch 손실 합
            epoch_corrects = 0  # epoch 정답 수

            # 학습하지 않았을 때의 검증 성능을 확인하기 위해 、epoch=0훈련 생략
            if (epoch == 0) and (phase == 'train'):
                continue

            # 데이터 로더에서 미니배치를 꺼내서 루프
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizer 초기화
                optimizer.zero_grad()

                # 순전파（forward）
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 손실 계산
                    _, preds = torch.max(outputs, 1)  # 예측

                    # 훈련 시에는 오차 역전피법
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 결과 계산
                    epoch_loss += loss.item() * inputs.size(0)  # loss의 합계 갱신
                    # 정답 수의 합계 갱신
                    epoch_corrects += torch.sum(preds == labels.data)

            # 에폭별 손실과 정답률 표시
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

In [26]:
# 学習・検証を実行する
num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 장치： cuda:0
Epoch 1/2
-------------


100%|██████████| 5/5 [00:22<00:00,  4.52s/it]


val Loss: 0.7702 Acc: 0.4444
Epoch 2/2
-------------


100%|██████████| 8/8 [00:40<00:00,  5.01s/it]


train Loss: 0.5033 Acc: 0.7284


100%|██████████| 5/5 [00:11<00:00,  2.31s/it]

val Loss: 0.1795 Acc: 0.9608


In [27]:
# 파이토치 네트워크 파라미터 저장
save_path = './weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [28]:
# PyTorch네트워크 파라미터 로드
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU 상에 저장된 가중치를 CPU로 로드할경우
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)

<All keys matched successfully>